In [1]:
import torch
from torch.autograd import Variable
import numpy as np

from pybullet_environment import RobotEnvironment
from Baselines.trajopt_torch import TrajOpt

device = "cuda" if torch.cuda.is_available() else "cpu"

pybullet build time: Jul 17 2023 13:38:55


In [2]:
env = RobotEnvironment(gui = False)
trajopt = TrajOpt(env, num_joints = 7, traj_len = 128)

argv[0]=


In [4]:
start_joints = np.array([-1.9300085,  1.3141115,  2.1577177, -1.5229155, -0.7427585, 1.4192199, -1.3984755])
goal_pose = np.array([0.5,  0.01,  0.4, 0.1, 0, 0, 0.0008])

trajopt.initialize_problem(start_joints, goal_pose)

In [49]:
trajopt.solve(epochs = 80000)

In [50]:
print("Total Cost =", trajopt.objective().detach().numpy(force = True))
print("Length Cost =", trajopt.length_cost(trajopt.theta).detach().numpy(force = True))
print("Goal Cost =", trajopt.goal_cost(trajopt.theta).detach().numpy(force = True))

Total Cost = 0.27122230234839734
Length Cost = 0.26209789089278585
Goal Cost = 0.009124411455611475


In [42]:
print(trajopt.theta[-10:, :])

tensor([[ 1.6034e-01,  1.0287e+00,  1.1400e-01,  5.5595e-01,  1.4385e-01,
          5.7853e-01,  7.4982e-01],
        [ 1.4746e-01,  1.0500e+00,  1.0011e-01,  5.6137e-01,  1.2908e-01,
          5.8133e-01,  7.5769e-01],
        [ 1.3442e-01,  1.0717e+00,  8.6085e-02,  5.6716e-01,  1.1416e-01,
          5.8411e-01,  7.6562e-01],
        [ 1.2123e-01,  1.0938e+00,  7.1936e-02,  5.7335e-01,  9.9098e-02,
          5.8687e-01,  7.7360e-01],
        [ 1.0790e-01,  1.1163e+00,  5.7681e-02,  5.7993e-01,  8.3916e-02,
          5.8959e-01,  7.8162e-01],
        [ 9.4449e-02,  1.1392e+00,  4.3336e-02,  5.8690e-01,  6.8634e-02,
          5.9227e-01,  7.8968e-01],
        [ 8.0902e-02,  1.1625e+00,  2.8919e-02,  5.9428e-01,  5.3268e-02,
          5.9489e-01,  7.9776e-01],
        [ 6.7273e-02,  1.1862e+00,  1.4447e-02,  6.0206e-01,  3.7839e-02,
          5.9745e-01,  8.0585e-01],
        [ 5.3579e-02,  1.2102e+00, -6.2368e-05,  6.1026e-01,  2.2366e-02,
          5.9994e-01,  8.1395e-01],
        [ 

In [51]:
final_traj = trajopt.theta.numpy(force=True)
np.save("final_traj.npy", final_traj)

In [20]:
zero_grad = torch.zeros((128, 7)).to(device)

print(trajopt.objective(zero_grad[1:, :]))
print(trajopt.length_cost(trajopt.theta))
print(trajopt.goal_cost(trajopt.theta))
#print(trajopt.theta)

tensor(0., device='cuda:0')
tensor(160.7867, device='cuda:0')
tensor(4.3555, device='cuda:0', dtype=torch.float64)


In [14]:
theta = torch.tensor([-1.9300085,  1.3141115,  2.1577177, -1.5229155, -0.7427585, 1.4192199, -1.3984755]).to(device)
theta = torch.rand((10, 7)).to(device) 
grad_theta = torch.zeros_like(theta, requires_grad = True).to(device)

goal_pose = np.array([0.5,  0.01,  0.4, 0.1, 0, 0, 0.0008])
goal_transform = torch.tensor(env.pose_to_transformation(goal_pose)).to(device)

In [5]:
print(grad_theta.requires_grad)

True


In [8]:
for i in range(500):

    grad_theta.grad = None

    new_theta = theta + grad_theta

    current_transform = trajopt.forward_kinematics(new_theta[-1, :]).to(device)

    goal_error = torch.sum((goal_transform - current_transform) ** 2)

    #grad = torch.autograd.grad(goal_error, grad_theta)
    goal_error.backward()

    with torch.no_grad():
        grad_theta -= 0.002*grad_theta.grad

print(goal_error)


tensor(0.1070, device='cuda:0', dtype=torch.float64, grad_fn=<SumBackward0>)


In [274]:
new_theta = theta + grad_theta

current_transform = trajopt.forward_kinematics(new_theta)

print(current_transform)
print(goal_transform)
print(new_theta)

tensor([[ 9.9999e-01,  6.7118e-04, -5.1359e-03,  5.1666e-01],
        [ 5.8642e-04, -9.9986e-01, -1.6490e-02,  8.7427e-03],
        [-5.1463e-03,  1.6486e-02, -9.9985e-01,  4.2422e-01],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],
       grad_fn=<MmBackward0>)
tensor([[ 1.0000,  0.0000,  0.0000,  0.5000],
        [ 0.0000, -0.9999, -0.0160,  0.0100],
        [ 0.0000,  0.0160, -0.9999,  0.4000],
        [ 0.0000,  0.0000,  0.0000,  1.0000]], dtype=torch.float64)
tensor([-1.7249,  0.4717,  1.9645, -1.9570, -0.4539,  1.7351,  1.1511],
       grad_fn=<AddBackward0>)


In [44]:
env.client_id.disconnect()